<a href="https://colab.research.google.com/github/mzsmarcos/projeto_pipelineETL/blob/main/Etapa_5_jun%C3%A7%C3%A3o_parte_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Junção Parte 1**

Nesse notebook estão os códigos da primeira parte de junção dos arquivos do site da Receita Federal:

Sequência das junções: Ordenada por criticidade

1 - df_empresas_brasil_1 = df_estab + df_socios

2 - df_empresas_brasil_1 + df_empresas

3 - df_empresas_brasil_1 + df_simples



Configuração do PySpark -  instalar o PySpark no Colab

In [ ]:
# Instalar o PySpark
!pip install pyspark


Executar a instalação do PySpark e configurar o ambiente Spark.

Isso é feito definindo algumas variáveis de ambiente para garantir que o Spark funcione corretamente no Colab.

In [ ]:
# Instalar Java 8
!apt-get install openjdk-8-jdk -y

# Baixar o Apache Spark
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

# Extrair o Apache Spark
!tar xf spark-3.5.0-bin-hadoop3.tgz

# Definir variáveis de ambiente
import os # Módulo para interagir com o sistema operacional, como manipulação de arquivos e diretórios
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

# Instalar as bibliotecas PySpark e Findspark
!pip install -q pyspark==3.5.0 findspark



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libice-dev librsvg2-common
  libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre
  openjdk-8-jre-headless x11-utils
Suggested packages:
  gvfs libice-doc libsm-doc libxt-doc openjdk-8-demo openjdk-8-source visualvm libnss-mdns
  fonts-nanum fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei
  fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libice-dev librsvg2-common
  libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-

In [ ]:
# Verificar se o arquivo foi baixado
!ls -lh


total 382M
drwxr-xr-x  1 root root 4.0K Mar 13 13:31 sample_data
drwxr-xr-x 13 1000 1000 4.0K Sep  9  2023 spark-3.5.0-bin-hadoop3
-rw-r--r--  1 root root 382M Sep  9  2023 spark-3.5.0-bin-hadoop3.tgz


Criar a Sessão Spark

In [ ]:
# Importar as bibliotecas
import findspark # Permite localizar e configurar o Spark no ambiente, garantindo que o PySpark possa ser usado
findspark.init("/content/spark-3.5.0-bin-hadoop3")

from pyspark.sql import SparkSession  # Importa a classe SparkSession para criar uma sessão do Spark e interagir com DataFrames

# Criar a sessão Spark
spark = SparkSession.builder \
    .appName("PySparkExample") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

# Verificar se a sessão foi criada corretamente
print("Versão do Spark:", spark.version)


Versão do Spark: 3.5.0


In [ ]:
# Os
import os

In [ ]:
# Montando o Google Drive no Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Identificação dos arquivos parquet no diretório

#Lista arquivos do diretório

files = os.listdir("/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/parquet")

# Exibe os arquivos
for file in files:
    print(file)



cnae.parquet
estab_unificado.parquet
empresas_unificado.parquet
motivos.parquet
municipios.parquet
natureza.parquet
paises.parquet
qualificacao.parquet
simples.parquet
socios_unificado.parquet
socios_agrupado.parquet


In [ ]:
# Validação das chaves de junção

from pyspark.sql.functions import col

def verificar_duplicatas(df, chave):
    total_linhas = df.count()
    valores_unicos = df.select(chave).distinct().count()

    print(f"🔍 Verificando {chave} em dataframe com {total_linhas} linhas...")
    print(f"  → Valores únicos: {valores_unicos}")

    if valores_unicos < total_linhas:
        print(f"  ⚠️ Há {total_linhas - valores_unicos} duplicatas na chave '{chave}'!\n")
    else:
        print(f"  ✅ Nenhuma duplicata encontrada na chave '{chave}'!\n")

# Rodar para cada dataframe e chave
verificar_duplicatas(empresas_unificado, "cnpj_basico")
verificar_duplicatas(estab_unificado, "cnpj_basico")
verificar_duplicatas(simples, "cnpj_basico")

# Para df_socios, precisamos verificar múltiplas chaves
for chave in ["cnpj_basico", "array_nome_socio_pf_pj", "array_cpf_cnpj_socio", "array_cpf_representante_legal", "array_nome_representante"]:
    verificar_duplicatas(socios_agrupado, chave)





🔍 Verificando cnpj_basico em dataframe com 60959932 linhas...
  → Valores únicos: 60959931
  ⚠️ Há 1 duplicatas na chave 'cnpj_basico'!

🔍 Verificando cnpj_basico em dataframe com 64017368 linhas...
  → Valores únicos: 60959931
  ⚠️ Há 3057437 duplicatas na chave 'cnpj_basico'!



In [ ]:
# Junçao dos arquivos

from pyspark.sql.functions import lit
from pyspark.sql.utils import AnalysisException
import os
import traceback

# Caminho do diretório no Google Drive
drive_directory = "/content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/parquet"

# Lista de arquivos parquet
arquivos_parquet = [
    "estab_unificado.parquet",
    "empresas_unificado.parquet",
    "simples.parquet",
    "qualificacao.parquet",
    "natureza.parquet",
    "municipios.parquet",
    "motivos.parquet",
    "paises.parquet",
    "cnae.parquet",
    "socios_agrupado.parquet"
]

# Configurações otimizadas para PySpark
spark.conf.set("spark.sql.parquet.compression.codec", "snappy")
spark.conf.set("spark.sql.parquet.mergeSchema", "false")
spark.conf.set("spark.sql.parquet.filterPushdown", "true")
spark.conf.set("spark.sql.files.maxPartitionBytes", "128m")
spark.conf.set("spark.sql.shuffle.partitions", "200")

# Função para verificar se o arquivo parquet é válido
def verificar_arquivo_parquet(caminho):
    """Verifica se um arquivo parquet pode ser lido corretamente."""
    try:
        # Tenta ler apenas o esquema do arquivo, sem carregar dados
        schema = spark.read.option("mergeSchema", "false").parquet(caminho).schema
        print(f"Esquema do arquivo validado: {caminho}")
        return True
    except Exception as e:
        print(f"Erro ao verificar esquema do arquivo {caminho}: {e}")
        return False

# Dicionário para armazenar os dataframes
dataframes = {}

total_linhas = 0  # Variável para armazenar o total de linhas

# Loop para ler os arquivos parquet com tratamento de erros melhorado
for arquivo in arquivos_parquet:
    caminho_completo = f"{drive_directory}/{arquivo}"
    df_name = arquivo.split(".")[0]

    if os.path.exists(caminho_completo):
        print(f"Verificando arquivo: {caminho_completo}")

        # Verifica se o arquivo é válido antes de tentar carregá-lo
        if verificar_arquivo_parquet(caminho_completo):
            try:
                # Tenta ler o arquivo sem contar as linhas inicialmente
                print(f"Carregando arquivo: {caminho_completo}")
                df = spark.read.option("mergeSchema", "false").parquet(caminho_completo)

                # Verifica se o DataFrame pode ser processado
                try:
                    # Tenta executar uma operação simples no DataFrame
                    df.dtypes  # Isso não executa um job Spark, apenas recupera metadados
                    print(f"Arquivo {arquivo} carregado com sucesso.")

                    # Só cache se o DataFrame for usado nos joins
                    if df_name == "estab_unificado" or df_name in ["socios_agrupado", "empresas_unificado", "simples"]:
                        df = df.cache()
                        print(f"DataFrame {df_name} colocado em cache.")

                    # Contagem de linhas com tratamento de erros
                    try:
                        print(f"Contando linhas para {df_name}...")
                        num_linhas = df.count()
                        total_linhas += num_linhas
                        print(f"Arquivo {arquivo} tem {num_linhas} linhas.")
                    except Exception as e:
                        print(f"Erro ao contar linhas do arquivo {arquivo}: {e}")
                        print(f"Detalhes do erro: {traceback.format_exc()}")
                        # Continua sem contar as linhas
                        num_linhas = 0

                    dataframes[df_name] = df

                except Exception as e:
                    print(f"Erro ao processar o DataFrame {df_name}: {e}")
                    print(f"Detalhes do erro: {traceback.format_exc()}")
                    dataframes[df_name] = None

            except Exception as e:
                print(f"Erro ao ler o arquivo {arquivo}: {e}")
                print(f"Detalhes do erro: {traceback.format_exc()}")
                dataframes[df_name] = None
        else:
            print(f"Arquivo {arquivo} não pôde ser validado.")
            dataframes[df_name] = None
    else:
        print(f"Arquivo não encontrado: {caminho_completo}")
        dataframes[df_name] = None

print(f"Total estimado de linhas nos arquivos carregados: {total_linhas}")

# Verifica se o DataFrame principal foi carregado
df_empresas_brasil_1 = dataframes.get("estab_unificado")

if df_empresas_brasil_1 is not None:
    print("DataFrame estab_unificado carregado com sucesso. Prosseguindo com os joins.")

    # Lista de dataframes a serem unidos
    dataframes_a_unir = ["socios_agrupado", "empresas_unificado", "simples"]
    total_joins = len(dataframes_a_unir)

    for i, df_name in enumerate(dataframes_a_unir):
        df_atual = dataframes.get(df_name)
        if df_atual is not None:
            try:
                # Verifica se a coluna de join existe
                colunas_df_atual = df_atual.columns
                if "cnpj_basico" in colunas_df_atual:
                    try:
                        # Executa o join com tratamento de erros
                        print(f"Iniciando join com {df_name}...")
                        df_empresas_brasil_1 = df_empresas_brasil_1.join(df_atual, ["cnpj_basico"], "left")

                        progresso = ((i + 1) / total_joins) * 100
                        print(f"Join com {df_name} concluído. Progresso: {progresso:.2f}%")

                        # Libera memória do DataFrame que já foi usado no join
                        try:
                            df_atual.unpersist()
                            print(f"Memória liberada para o DataFrame {df_name}")
                        except:
                            print(f"Não foi possível liberar a memória para {df_name}")

                        dataframes[df_name] = None

                    except Exception as e:
                        print(f"Erro ao realizar join com {df_name}: {e}")
                        print(f"Detalhes do erro: {traceback.format_exc()}")
                else:
                    print(f"DataFrame {df_name} não contém a coluna 'cnpj_basico'. Join ignorado.")
                    print(f"Colunas disponíveis: {colunas_df_atual}")
            except Exception as e:
                print(f"Erro ao processar o DataFrame {df_name} para join: {e}")
                print(f"Detalhes do erro: {traceback.format_exc()}")
        else:
            print(f"DataFrame {df_name} não encontrado, join ignorado.")

    try:
        # Usa um método seguro para tratar dados faltantes
        print("Tratando valores nulos...")
        df_empresas_brasil_1 = df_empresas_brasil_1.na.fill("Não aplicado")
        print("Tratamento de valores nulos concluído.")

        # Caminho de saída
        output_path = f"{drive_directory}/df_empresas_brasil_1"

        # Determina o número de partições
        num_colunas = len(df_empresas_brasil_1.columns)
        linhas_estimadas = df_empresas_brasil_1.count()  # Conta as linhas reais

        # Ajusta o número de partições com base no tamanho estimado
        tamanho_estimado = linhas_estimadas * num_colunas * 8  # Estimativa grosseira em bytes
        tamanho_partição = 128 * 1024 * 1024  # 128MB por partição
        partition_count = max(1, min(int(tamanho_estimado / tamanho_partição), 200))

        print(f"Iniciando salvamento com {partition_count} partições...")

        # Tenta salvar o resultado com tratamento de erros
        try:
            df_empresas_brasil_1.repartition(partition_count) \
                .write \
                .mode("overwrite") \
                .option("compression", "snappy") \
                .option("parquet.block.size", "134217728") \
                .option("parquet.page.size", "1048576") \
                .option("parquet.enable.dictionary", "true") \
                .parquet(output_path)

            print(f"Salvamento concluído com {partition_count} partições.")
            print(f"Arquivos salvos com compressão Snappy em: {output_path}")
        except Exception as e:
            print(f"Erro ao salvar o resultado: {e}")
            print(f"Detalhes do erro: {traceback.format_exc()}")

            # Tenta um salvamento mais simples
            print("Tentando salvar com configurações básicas...")
            try:
                df_empresas_brasil_1.write.mode("overwrite").parquet(output_path)
                print("Salvamento básico concluído.")
            except Exception as e2:
                print(f"Também falhou o salvamento básico: {e2}")
                print(f"Detalhes do erro: {traceback.format_exc()}")

    except Exception as e:
        print(f"Erro ao processar o DataFrame final: {e}")
        print(f"Detalhes do erro: {traceback.format_exc()}")

    # Libera memória dos DataFrames lidos
    for df_name, df in dataframes.items():
        if df is not None:
            try:
                df.unpersist()
                print(f"Memória liberada para o DataFrame {df_name}")
            except:
                pass
else:
    print("DataFrame estab_unificado não encontrado. Verifique a leitura dos arquivos.")

Verificando arquivo: /content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/parquet/estab_unificado.parquet
Esquema do arquivo validado: /content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/parquet/estab_unificado.parquet
Carregando arquivo: /content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/parquet/estab_unificado.parquet
Arquivo estab_unificado.parquet carregado com sucesso.
DataFrame estab_unificado colocado em cache.
Contando linhas para estab_unificado...
Arquivo estab_unificado.parquet tem 64017368 linhas.
Verificando arquivo: /content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/parquet/empresas_unificado.parquet
Esquema do arquivo validado: /content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/parquet/empresas_unificado.parquet
Carregando arquivo: /content/drive/MyDrive/Projeto_ETL_Trust_Works/Dados_abertos/parquet/empresas_unificado.parquet
Arquivo empresas_unificado.parquet carregado com sucesso.
DataFrame empresas_unificado colocado em c

In [ ]:
# Mostra algumas linhas do DataFrame final para inspeção
df_empresas_brasil_1.show(5)

+-----------+----------+-------+----------------+--------------------+-----------------+------------------+---------------------+---------------+------------+---------------------+---------------------+----------------------+--------------+--------------------+------+--------------------+-----------------+--------+---+---------+------------+------------+------------+------------+------------+------------+--------------------+-----------------+----------------------+------------------+----------------------+--------------------+---------------------+----------------------------+--------------------+-----------------------------+------------------------+-----------------------------+----------------------+----------------------+--------------------+----------------+----------------------------+--------------+-----------------+--------------------+-------------+------------------+-----------------+------------+--------------+-------------+
|cnpj_basico|cnpj_ordem|cnpj_dv|id_matriz_filial|